Chapter 1 - Python Data Model
=============================

The python data model follows a pattern that is usually different from the ones that we are
used to in other programming languages.  To be able to extend the functionality of your custom
types you should provide implementations of "_dunder methods_" and use python methods to call
instead of having your own methods/implementations.

    len(my_object)
    
    # instead of
    
    my_object.len()

One thing that I want to bring up that has probably not been covered yet, is what they call the `Zen of Python` or the basic rules that should be followed when writing python code.  There are quite a few useful rules and we should go over them, luckily python has the list built-in to the language.

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


Alright, with those rules out of the way, lets start with our first test.

The first thing that we are going to create is a custom gaming deck that the book calls a `FrenchDeck`, you
can see that we can create a `namedtuple` which is a custom data store that can be similar to structs in some
languages, but also gives you the nice tuple functionality.

We are also demonstrating the use of `__len__` and `__getitem__` dunder methods.  I also added the `__repr__`
method to make the output in the jupyter notebook a little cleaner.

In [2]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __repr__(self):
        return f'FrenchDeck()'

In [3]:
from itertools import islice
from random import choice

my_deck = FrenchDeck()

# Just demonstrate the __repr__
print('1:', my_deck)

# This works because of __getitem__
print('2:', my_deck[2])

# This works because of __len__
print('3:', len(my_deck))

# This works because of __getitem__, could also work through __iter__
for c in islice(my_deck, 2):
    print('4:', c)
    
# This works because of __len__ and __getitem__
print('5:', choice(my_deck))

# This works because of our __getitem__ implementation that supports a 'slice' position
print('6:', my_deck[:2])

1: FrenchDeck()
2: Card(rank='4', suit='spades')
3: 52
4: Card(rank='2', suit='spades')
4: Card(rank='3', suit='spades')
5: Card(rank='Q', suit='clubs')
6: [Card(rank='2', suit='spades'), Card(rank='3', suit='spades')]


From the above example we can see that by using certain "dunder methods" we can have our object interact with the different functions that are built-in, or even created from libraries, very easily.  The interesting part is which of the different methods are used in which cases, and even more importantly what the content of those requests are.

In this next section there is an example class that has 3 different dunder methods used:
* `__init__`
* `__len__`
* `__getitem__`

To start lets show what is passed in and to which method when we use different python requests.

In [7]:
class ShowMethodCalls:
    def __init__(*args, **kwargs):
        print('Init: ', args, kwargs)
        
    def __len__(*args, **kwargs):
        print('Length: ', args, kwargs)
        return 0

    def __getitem__(*args, **kwargs):
        print('GetItem: ', args, kwargs)
        
x = ShowMethodCalls()
print('Len test')
len(x)

print('Get index')
x[0]
x[-1]
x['test']
x['test', 'other']

print('Slice index')
x[:1]
x[:]

Init:  (<__main__.ShowMethodCalls object at 0x7f7e343e6bc0>,) {}
Len test
Length:  (<__main__.ShowMethodCalls object at 0x7f7e343e6bc0>,) {}
Get index
GetItem:  (<__main__.ShowMethodCalls object at 0x7f7e343e6bc0>, 0) {}
GetItem:  (<__main__.ShowMethodCalls object at 0x7f7e343e6bc0>, -1) {}
GetItem:  (<__main__.ShowMethodCalls object at 0x7f7e343e6bc0>, 'test') {}
GetItem:  (<__main__.ShowMethodCalls object at 0x7f7e343e6bc0>, ('test', 'other')) {}
Slice index
GetItem:  (<__main__.ShowMethodCalls object at 0x7f7e343e6bc0>, slice(None, 1, None)) {}
GetItem:  (<__main__.ShowMethodCalls object at 0x7f7e343e6bc0>, slice(None, None, None)) {}


We can see some interesting things, the first is understanding that the first argument will always be a pointer to the instance itself, which is why we put self in there. (**NOTE:** for other languages this is automatically handled and put into the `this` variable).

The next interesting item is what is passed to `__getitem__`.  It is still just one extra argument after `self`, but in this case it changes the type depending on what you are passing. You can supply anything inside the brackets that is valid python and it will pass the result as the type supplied, which is most odd when you pass a slice.  This actually creates an instance of the `slice` type which gives you the start, stop and increment fields.

Lets not compare when `__getitem__` is used vs `__iter__` if both are supplied.Cython-0.29.28

In [19]:
class NoIter:
    def __len__(self):
        print('Len')
        return 2
    
    def __getitem__(self, position):
        print('GetItem:', position)
        return 0

class WithIter(NoIter):
    def __iter__(self, *args, **kwargs):
        count = 0
        while True:
            print(f'Iter: {count}', args, kwargs)
            yield count
            count += 1
            
print('Sample with only len, getitem')
x = NoIter()
x[0]
x[:2]
for _ in islice(x, 2):
    pass
for _ in x:
    break
    

print('\nSample with iter and len, getitem')
y = WithIter()
y[0]
y[:2]
for _ in islice(y, 2):
    pass
for _ in y:
    break

Sample with only len, getitem
GetItem: 0
GetItem: slice(None, 2, None)
GetItem: 0
GetItem: 1
GetItem: 0

Sample with iter and len, getitem
GetItem: 0
GetItem: slice(None, 2, None)
Iter: 0 () {}
Iter: 1 () {}
Iter: 0 () {}


So you can see that it will fallback to `__getitem__` if the `__iter__` is not available.  I believe this is true in almost 100% of the cases if not 100% (can't think of a situation it wouldn't work in). One really important thing to call out though, the way that `__iter__` works is by returning an object that provides the special method `__next__` which is used by the python interpreter when iterating.  The above code could actually be changed to be the following.

In [20]:
class IterAndNext:
    def __iter__(self):
        class MyNext:
            count = 0
            def __next__(self):
                print(f'Iter: {self.count}')
                orig = self.count
                self.count += 1
                return orig
        return MyNext()

x = IterAndNext()
for _ in islice(x, 2):
    pass
for _ in x:
    break

Iter: 0
Iter: 1
Iter: 0


## String vs Repr

There are two different ways that we can represent our class instance as a string, these are handled by the `__repr__` and `__str__` dunder methods.  One of the interesting things is the order of operation that they are implemented in as there is a fallback that takes places when one of the methods is not created.

In [21]:
class WithStr:
    def __str__(self):
        return f'Str: WithStr'
    
class WithRepr:
    def __repr__(self):
        return f'Repr: WithRepr'
    
class WithBoth:
    def __str__(self):
        return f'Str: WithBoth'
    def __repr__(self):
        return f'Repr: WithBoth'
    
x = WithStr()
print(x)
display(x)

x = WithRepr()
print(x)
display(x)

x = WithBoth()
print(x)
display(x)

Str: WithStr


Repr: WithRepr


Repr: WithRepr

Str: WithBoth


Repr: WithBoth

Given the above it is often best to either implement just `__repr__` or both of them.  While `__str__` does fallback to `__repr__` it is important to understand the use of both of these methods.  The `__repr__` should be used to display an output that if pasted in a repr would recreate the instance of the class.

```
class MyClass:
    def __init__(self, name):
        ...
    def __repr__(self):
        return f'MyClass({name})'
```

On the other hand, `__str__` should be able to provide useful information for a consumer of the data, which may or may not be an engineer.  While a fallback to `__repr__` is useful, you need to be careful to not introduce sensitive information that may be useful while developing, but should never get out in production. 

## Collection API

One other section that is covered is the collection api and describing the inheritance structure of the API.  It is important to note that you actually don't need to inherit from anything to use the api, you just need to implement the correct dunder methods and your custom class will act like that portion of a collection.

Let's try a couple of different APIs out.

In [27]:
class DoIHaveIt:
    def __contains__(self, *args, **kwargs):
        print('Contains:', args, kwargs)
        return False

class ContainsNoContain:
    def __getitem__(self, position):
        print('Getitem:', position)
        if position > 2:
            raise IndexError()
        return 0
    
x = DoIHaveIt()
10 in x
'a' in x
('a', 'b') in x

y = ContainsNoContain()
0 in y

Contains: (10,) {}
Contains: ('a',) {}
Contains: (('a', 'b'),) {}
Getitem: 0


True

It is important to check though as some of the more "advanced" collection classes do have other methods that have to be implemented for example a `Map` needs to implement `keys` and `values`.

## Operator Overloading

Python, unlike Java, does support the notion of operator overloading and "surprise, surprise" it is handled through dunder methods.  The interesting part of python is that it supports operations that might be different from most other languages.  Some of these include `__abs__`, `__round__`, `__pow__` and others. The potential downside is that you can also overload operators to not function in the same way (say a different impl for `__add__` vs `__radd__` or `__iadd__`.  Let's look at some examples.

In [43]:
class MyBadMath:
    def __init__(self, num):
        self.num = num
        
    def __add__(self, other):
        return MyBadMath(self.num + int(other))
    
    def __radd__(self, other):
        return MyBadMath(self.num - int(other))
    
    def __iadd__(self, amount):
        return MyBadMath(self.num + self.num)
    
    def __repr__(self):
        return f'MyBadMath({self.num})'

class OtherMath:
    def __add__(self, right):
        print('My add')
        if type(right) != int:
            return NotImplemented
        return self
    
    def __radd__(self, right):
        print('My radd')
        return right
    
    def __int__(self):
        return 10

x = MyBadMath(3)
print(x + '20')
print(x + 10)
x += 10; print(x)
print(10 + x)

y = OtherMath()
print(y + x)

MyBadMath(23)
MyBadMath(13)
MyBadMath(6)
MyBadMath(-4)
My add
MyBadMath(-4)


Of course you don't actually have to implement all the math methods to still get some common functionality, unlike C++.

In [51]:
class PartialMath:
    def __init__(self, num):
        self.num = num
        
    def __add__(self, other):
        return PartialMath(self.num + other)
    
    def __repr__(self):
        return f'PartialMath({self.num})'
    
#display(dir(PartialMath(2)))

x = PartialMath(3)
print(x + 10)
x += 1
print(x)
print(10 + x)

PartialMath(13)
PartialMath(4)


TypeError: unsupported operand type(s) for +: 'int' and 'PartialMath'

## Bool operator

Python has the notion of _truthy_ which means that anything can be used in a logic evaluation.  There are some important rules as to what is consider false with most objects.  The reason we call out false first is because the list is smaller and anything not on the list will default to true.

* Numerics who value is equivelant to 0
  * 0, 0.0, etc
* Empty sequences
  * '', [], (), {}, etc
* None
* False

This means that instances of a custom class would be considered `True`, although that is not always what you want. To be able to adjust how your custom class responds you can overload the `__bool__` method.  When you do this you do need to return an actual boolean instance in the response.

In [52]:
class MyEvens:
    def __init__(self, num):
        self.num = num
    def __bool__(self):
        return self.num % 2 == 0
    
print(f'2', bool(MyEvens(2)))
print(f'10241', bool(MyEvens(10241)))

2 True
10241 False


## Practice

For this next section we are going to use what we have learned about the object model to complete the following code segment. This one is pretty simple we are going to implement Fibonacci from a class that follows the data model. There is some driver code that you will run to verify your implementation works as expected.

In [ ]:
class FibonacciYield:
    """
    This class should be able to act as an iterable that will always start the sequence
    over when a new iterator is created
    """
    pass


In [ ]:
#
# Sample Answer
#

class SampleFibonacciYield:
    """
    This class should be able to act as an iterable that will always start the sequence
    over when a new iterator is created
    """
    def __iter__(self):
        previous = 1
        yield previous
        current = 1
        yield current
        while True:
            current, previous = current + previous, current
            yield current

In [ ]:
x = FibonacciYield()

first_ten = [n[0] for n in zip(x, range(10))]
expected_ten = [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

assert first_ten == expected_ten

In [ ]:
class FibonacciNext:
    """
    This class should be able to act as an interable that will continue the sequence when
    a new instance of the iterator is requested
    """
    pass

In [ ]:
class SampleFibonacciNext:
    """
    This class should be able to act as an interable that will continue the sequence when
    a new instance of the iterator is requested
    """
    def __init__(self):
        self.current = 1
        self.previous = 1
        self.count = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        #print('In next', self.count)
        orig_count = self.count
        self.count += 1
        
        if orig_count == 0:
            return self.previous
        if orig_count == 1:
            return self.current
        
        self.current, self.previous = self.current + self.previous, self.current
        return self.current
        

In [ ]:
x = FibonacciNext()

first_five = [n[0] for n in zip(x, range(5))]
expected_five = [1, 1, 2, 3, 5]
assert first_five == expected_five

next_five = [n[0] for n in zip(x, range(5))]
expected_next = [13, 21, 34, 55, 89]
assert next_five == expected_next
